In [1]:
from datetime import datetime
from IPython.display import HTML
from IPython.display import Markdown as md
import ipywidgets as widgets
import time

BUSCA = ['Engenheiro','Engineer','Instrumentacao','Instrumentation','Automacao','Automation']

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
import requests
from bs4 import BeautifulSoup
import wget
from IPython.display import HTML
from IPython.display import Markdown as md
import ipywidgets as widgets

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

def vagas(tags):
    paginas = []
    for a in tags:
        x=a.find('a', href=True)
        link = x['href']
        texto = x.text.rstrip("\n")
        titulo = texto.strip()
        descricao = a.find('p').getText().strip()
        x = a.find("span", {"class": "vaga-local"})
        if not isinstance(x, (type(None))):
            local = x.getText().strip()
        else:
            local = ''
        x = a.find("span", {"class": "data-publicacao"})
        if not isinstance(x, (type(None))):
            data = x.getText().strip()
        else:
            data = ''
        paginas.append([titulo, 'https://www.vagas.com.br/'+link, local, data, descricao])
    return paginas

def gera_vagas(busca):
    URL = 'https://www.vagas.com.br/vagas-de-'+busca+'?ordenar_por=mais_recentes'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tags = soup.select('li[class="vaga odd"]')
    list1 = vagas(tags)
    paginas = []
    tags = soup.select('li[class="vaga even"]')
    list2 = vagas(tags)
    paginas = concatena_listas(list1,list2)
    return imprimir([row[0] for row in paginas],[row[1] for row in paginas],[row[2] for row in paginas],[row[3] for row in paginas])


##########################################################
def catho(tags):
    """global paginas
    global link
    global a
    global titulo
    global descricao
    global local
    global data"""
    paginas = []
    for a in tags:
        #print(len(a),type(a), a.text)
        #x = a.find("a", {"class": "viewVagaAction"})
        x = a.find("a", {"class": "viewVagaAction"})
        if not isinstance(x, (type(None))):
            link = x['href']
        else:
            link = '' 
        x = a.find("a", {"class": "viewVagaAction"})
        if not isinstance(x, (type(None))):
            titulo = x['title']
        else:
            titulo = ''
        x = a.find("div", {"class": "descriptionComplete"})
        if not isinstance(x, (type(None))):
            descricao = x.text.strip()
        else:
            descricao = ''
        local = ''
        x = a.find("time", {"class": "dataAnuncio"})
        if not isinstance(x, (type(None))):
            data = x.text.strip()
        else:
            data = ''
        paginas.append([titulo, link, local, data, descricao])
    return paginas
##########################################################
def gera_catho(busca):
    """global paginas
    global i
    global page
    global soup
    global tags
    global x"""
    paginas = []
    
    
    for i in range(1,5):
        #time.sleep(2)
        #URL = 'https://www.catho.com.br/vagas/'+busca+'/?q='+busca+'&pais_id=31&faixa_sal_id_combinar=1&perfil_id=1&order=dataAtualizacao&where_search=1&how_search=2&page='+str(i)
        #URL = 'https://www.catho.com.br/vagas/'+busca+'/?q='+busca+'&order=dataAtualizacao&page='+str(i)
        #page = requests.get(URL)
        #soup = BeautifulSoup(page.content, 'html.parser')
        #tags = soup.select('ul[class="gtm-class sc-fBuWsC eXuzGH"]')
        #print(len(tags),type(page))
        
        options = webdriver.ChromeOptions();
        options.add_argument("--user-data-dir=/home/eduardo/.config/google-chrome")
        options.add_argument('--start-maximized')
        driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
        print(i)
        URL = 'https://www.catho.com.br/vagas/'+busca+'/?q='+busca+'&pais_id=31&faixa_sal_id_combinar=1&perfil_id=1&order=dataAtualizacao&where_search=1&how_search=2&page='+str(i)
        driver.get(URL)        
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tags = soup.select('ul[id="listagemVagas"]')
        x=tags[0].find_all('li')
        paginas += catho(x)
        #driver.close()
    #return paginas
    return imprimir([row[0] for row in paginas],[row[1] for row in paginas],[row[2] for row in paginas],[row[3] for row in paginas])
    
##########################################################

def linkedin(tags):
    paginas = []
    for a in tags:
        link = a.find('a', href=True)['href']
        titulo = a.find("h3", {"class": "result-card__title job-result-card__title"}).text.strip()
        local = a.find("span", {"class": "job-result-card__location"}).text.strip()
        data = a.find("time", {"class": "job-result-card__listdate--new"})
        if isinstance(data, (type(None))):
            data=a.find("time", {"class": "job-result-card__listdate"}).text
        else:
            data = data.text
        #print("° "+titulo,"|", local,"|", data, '\n'+link)
        paginas.append([titulo, link, local, data])
    return paginas

def gera_linkedin(busca):
    #URL = 'https://www.linkedin.com/jobs/search/?f_E=4%2C5%2C6&geoId=106057199&keywords='+busca+'%20NOT%20Programador%20NOT%20devops%20NOT%20dados&location=Brasil&sortBy=DD'
    paginas = []
    paginacao = [0]
    for z in paginacao:
        URL= 'https://www.linkedin.com/jobs/search/?geoId=106057199&keywords='+busca+'%20NOT%20Programador%20NOT%20engenhariajob%20NOT%20devops%20NOT%20dados%20NOT%20seleta%20NOT%20desidy%20NOT%20pice%20NOT%20renner%20NOT%20Foodjobs.com.br&location=Brasil&sortBy=DD&start='+str(z)
        #print(URL)
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        tags = soup.select('ul[class="jobs-search__results-list"]')
        x=tags[0].find_all('li')
        paginas += linkedin(x)
    return imprimir([row[0] for row in paginas],[row[1] for row in paginas],[row[2] for row in paginas],[row[3] for row in paginas])

##########################################################

def concatena_listas(list1,list2):
    sums_list = []
    if len(list1)>len(list2):
        listA = list2
        listB = list1
    else:
        listA = list1
        listB = list2
    x = 0
    while x < len(listA):
        #print(x)
        sums_list.append(listA[x])
        sums_list.append(listB[x])
        x += 1
    for i in listB[x:]:
        sums_list.append(i)
    return sums_list


def imprimir(titulo, link, local, data):
    string1 ="<OL>"
    a = -1
    for tit in titulo:
        a=a+1
        if 'oje' in data[a] or 'hour' in data[a] or 'minut' in data[a]:
            data1 = '<font color="red"><b>'+data[a]+'</b></font>'
        else:
            data1 = data[a]
        string1 = string1 + "<LI>"
        string1 = string1 + '<a href="'+link[a]+'">'+tit+'</a>'+' | '+ local[a]+' | '+data1
        string1 = string1 + "</LI>"
    string1=string1+"</OL>"
    return HTML(string1)

# Pesquisa no Vagas e Linkedin


In [3]:
now = datetime.now()
current_time = now.strftime("%d/%m %H:%M")
#print("Current Time =", current_time)
md("## Última atualização %s"%(current_time))

## Última atualização 26/07 07:51

In [4]:
#children3 = [widgets.HTML(value=gera_catho(name).data, placeholder='', description='',) for name in BUSCA]
#print("Catho Ok", end =" ")
children2 = [widgets.HTML(value=gera_linkedin(name).data, placeholder='', description='',) for name in BUSCA]
print("Vagas Ok", end =" ")
children1 = [widgets.HTML(value=gera_vagas(name).data, placeholder='', description='',) for name in BUSCA]
print("Linkedin Ok")

Vagas Ok Linkedin Ok


In [5]:
#children  = children1 + children2 + children3
children  = children1
tab = widgets.Tab()
tab.children = children
for i in range(len(BUSCA)):
    tab.set_title(i, 'Vagas: '+BUSCA[i])
#for i in range(len(BUSCA), len(BUSCA)*2):
#    tab.set_title(i, 'Linkedin: '+BUSCA[i-len(BUSCA)])
#for i in range(len(BUSCA)*2,len(BUSCA)*3):
#    tab.set_title(i, 'Catho: '+BUSCA[i-len(BUSCA)*2])
tab

In [6]:
#children  = children1 + children2 + children3
children  = children2 
tab = widgets.Tab()
tab.children = children
for i in range(len(BUSCA)):
    tab.set_title(i, 'Linkedin: '+BUSCA[i])
tab

In [7]:
break

SyntaxError: 'break' outside loop (<ipython-input-7-6aaf1f276005>, line 1)

In [ ]:
x=tags[0].find_all('li')

In [ ]:
for a in x:
    print(a.find("time", {"class": "dataAnuncio"})['datetime'])

In [ ]:
from datetime import datetime

datetime_str = '09/19/18 13:55:26'

datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')

print(type(datetime_object))
print(datetime_object)  # printed in default format

In [ ]:
aaa = x[0].find("time", {"class": "dataAnuncio"})['datetime']

In [ ]:
gera_catho(BUSCA[2])

In [ ]:
soup.select('ul[id="listagemVagas"]')

In [ ]:
tags = soup.select('ul[class="gtm-class sc-fBuWsC eXuzGH"]')
x=tags[0].find_all('li')

In [ ]:
soup